In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
import pandas as pd
import visuals as vs
#Refer: http://scikit-learn.org/stable/modules/multiclass.html#multioutput-classification

In [9]:
features = pd.read_csv("F:\\ML\\MultiTarget\\features.csv")
target   = pd.read_csv("F:\\ML\\MultiTarget\\targets.csv")

In [10]:
print(features.shape, target.shape)

(1087, 102) (1087, 14)
